In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
data = pd.read_csv("orange_small_churn_data.txt")
data["target"]=pd.read_csv("orange_small_churn_labels.txt", header=None)
data.head(2)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,target
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1


In [3]:
#столбцы, в которых нет значений
# 16 численных
# 2 категории

fail=[]
for i in data.columns:
    if (data[i].isnull().sum() == len(data[i])): fail.append(i)
#Пока не знаю, как иначе отловить столбец из всех Nan
fail.append("Var118")

print (len(fail),fail)
data1=data.drop(fail, axis=1)


(19, ['Var8', 'Var15', 'Var20', 'Var31', 'Var32', 'Var39', 'Var42', 'Var48', 'Var52', 'Var55', 'Var79', 'Var141', 'Var167', 'Var169', 'Var175', 'Var185', 'Var209', 'Var230', 'Var118'])


In [4]:
# Заполним пропуски в численных данных
data1 = data1.fillna(data1.median(axis=0), axis=0)

In [5]:
numerical_columns=data1.columns[:173]
categorical_columns = data1.columns[173:211]

In [6]:
data1_describe = data1.describe(include=[object])

for c in categorical_columns:
    data1[c] = data1[c].fillna(data1_describe[c]['top'])
    
    # Теперь в данных нет пропусков


In [7]:
# Обработаем категориальные переменные
#Посмотрим количество разных категориальных типов в каждом признаке
for x in categorical_columns:
    print(x,len(set(data1[x].dropna().tolist())))
#Заметим, что есть некоторое количество признаков с очень большим числом значений - использовать OneHot, например, 
#на всех данных, неразумно

('Var191', 1)
('Var192', 354)
('Var193', 50)
('Var194', 3)
('Var195', 23)
('Var196', 4)
('Var197', 220)
('Var198', 3891)
('Var199', 4400)
('Var200', 13297)
('Var201', 2)
('Var202', 5542)
('Var203', 5)
('Var204', 100)
('Var205', 3)
('Var206', 21)
('Var207', 14)
('Var208', 2)
('Var210', 6)
('Var211', 2)
('Var212', 78)
('Var213', 1)
('Var214', 13297)
('Var215', 1)
('Var216', 1819)
('Var217', 12470)
('Var218', 2)
('Var219', 22)
('Var220', 3891)
('Var221', 7)
('Var222', 3891)
('Var223', 4)
('Var224', 1)
('Var225', 3)
('Var226', 23)
('Var227', 7)
('Var228', 30)
('Var229', 4)


In [8]:
#Обработаем бинарные признаки отдельно
# Признаки, в которых всего одно значение, информации не несут

binary_columns    = [c for c in categorical_columns if data1_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data1_describe[c]['unique'] > 2]
print binary_columns, nonbinary_columns

['Var201', 'Var208', 'Var211', 'Var218'] ['Var192', 'Var193', 'Var194', 'Var195', 'Var196', 'Var197', 'Var198', 'Var199', 'Var200', 'Var202', 'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var210', 'Var212', 'Var214', 'Var216', 'Var217', 'Var219', 'Var220', 'Var221', 'Var222', 'Var223', 'Var225', 'Var226', 'Var227', 'Var228', 'Var229']


In [9]:
#Значения бинарных признаков просто заменим на 0 и 1.

for c in binary_columns:
    top = data1_describe[c]['top']
    top_items = data1[c] == top
    data1.loc[top_items, c] = 0
    data1.loc[np.logical_not(top_items), c] = 1

In [10]:
#Посмотрим на результат:

data1[binary_columns].describe()

,Var201,Var208,Var211,Var218
count,40000,40000,40000,40000
unique,2,2,2,2
top,0,0,0,0
freq,39994,36946,32215,20813


In [11]:
data1_cat_code=pd.DataFrame()
data1_cat_nocode=pd.DataFrame()

In [12]:
# отберем столбцы, которые разумно кодировать
for x in nonbinary_columns:
    if (len(set(data1[x].dropna().tolist()))<31):
        data1_cat_code[x]=data1[x]
    else:
        data1_cat_nocode[x] = data1[x]

# и закодируем их через dummy
data1_cat_code = pd.get_dummies(data1_cat_code)

In [13]:
data1_cat_nocode.describe()

,Var192,Var193,Var197,Var198,Var199,Var200,Var202,Var204,Var212,Var214,Var216,Var217,Var220,Var222
count,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000
unique,354,50,220,3891,4400,13297,5542,100,78,13297,1819,12470,3891,3891
top,1GdOj1KXzC,RO12,0Xwj,fhk21Ss,r83_sZi,yP09M03,nyZz,RVjC,NhsEn4L,5zARyjR,mAjbk_S,gvA6,4UxGlow,catzS2D
freq,604,28817,3811,3557,750,20417,157,1464,23433,20417,3929,774,3557,3557


In [14]:
# значения на остальных заменим количеством вхождений
for x in data1_cat_nocode.columns:
    data1_cat_nocode[x] = data1_cat_nocode[x].map(data1.groupby(x).size())

In [15]:
data1_cat_nocode.head(2)

,Var192,Var193,Var197,Var198,Var199,Var200,Var202,Var204,Var212,Var214,Var216,Var217,Var220,Var222
0,222,1763,96,3,46,20417,2,1350,525,20417,161,2,3,3
1,298,5781,273,1,240,1,7,488,5125,1,76,1,1,1


In [16]:
#Нормируем данные
data_numerical = data1[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
count,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,...,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04
mean,-7.181172e-16,4.776732e-16,1.639159e-15,4.598092e-15,-8.683738e-16,-3.298528e-16,-2.907008e-16,1.197013e-15,-9.727774e-16,1.995660e-15,...,-2.655865e-16,-3.252162e-16,1.780880e-15,-1.521396e-15,8.531495e-16,5.037592e-16,-7.101524e-17,-5.161510e-12,3.661981e-16,-2.406861e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.589700e-02,-5.000000e-03,-1.516215e-02,-1.886168e-02,-5.910787e-02,-5.144314e-01,-1.149579e+00,-1.142457e+00,-6.458807e-02,-3.368723e-02,...,-5.108413e+00,-2.316928e-01,-3.222399e-01,-5.601663e-02,-3.324584e-02,-4.183833e-02,-6.217653e-01,-1.084422e+01,-4.703725e+00,-4.803691e+00
25%,-3.589700e-02,-5.000000e-03,-1.516215e-02,-1.886168e-02,-5.910787e-02,-2.787657e-01,-1.149579e+00,-2.194523e-02,-6.458807e-02,-3.368723e-02,...,-4.169863e-02,-2.316928e-01,-8.876006e-02,-5.601663e-02,-3.324584e-02,-4.183833e-02,-2.560508e-02,4.225332e-02,-4.373601e-03,-2.525782e-02
50%,-3.589700e-02,-5.000000e-03,-1.516215e-02,-1.886168e-02,-5.910787e-02,-1.680313e-01,3.117183e-02,-2.194523e-02,-6.458807e-02,-3.368723e-02,...,-4.169863e-02,-2.316928e-01,-8.876006e-02,-5.601663e-02,-3.324584e-02,-4.183833e-02,-2.560508e-02,4.225332e-02,-4.373601e-03,-2.525782e-02
75%,-3.589700e-02,-5.000000e-03,-1.516215e-02,-1.886168e-02,-5.910787e-02,1.936554e-02,3.117183e-02,-2.194523e-02,-6.458807e-02,-3.368723e-02,...,-4.169863e-02,-2.316928e-01,-8.876006e-02,-5.601663e-02,-3.324584e-02,-4.183833e-02,-2.560508e-02,4.225332e-02,-4.373601e-03,-2.525782e-02
max,1.226732e+02,1.999950e+02,1.836532e+02,1.077621e+02,4.922813e+01,5.293057e+01,4.754176e+00,1.277164e+02,7.161150e+01,5.855280e+01,...,2.475038e+01,2.043540e+01,2.550966e+01,8.410354e+01,1.781331e+02,8.513800e+01,1.350047e+02,2.324022e+01,6.617440e+00,8.249881e+01


In [17]:
#Соединяем все в одну таблицу
data2=pd.concat( [data_numerical, data1[binary_columns], data1_cat_code,data1_cat_nocode, data1["target"]], axis=1)

In [18]:
# Вырежем hold_out

msk = np.random.rand(len(data2)) < 0.8
train = data2[msk]
holdout = data2[~msk]
holdout.to_csv('hold_out.csv')

In [19]:
X = train.drop(('target'), axis=1)  
y = train['target']
feature_names = X.columns
print feature_names

Index([u'Var1', u'Var2', u'Var3', u'Var4', u'Var5', u'Var6', u'Var7', u'Var9',
       u'Var10', u'Var11',
       ...
       u'Var199', u'Var200', u'Var202', u'Var204', u'Var212', u'Var214',
       u'Var216', u'Var217', u'Var220', u'Var222'],
      dtype='object', length=370)


In [23]:
# Сначала кросс валидацией разобьем данные на обучение и тест

from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.33, random_state=42) 

# Проверим модели

# Ridge

In [27]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier()

#Проверка модели на кросс-валидации
scores = cross_validation.cross_val_score(clf, X, y, cv = 5)
scoresROCAUC =  cross_validation.

print ("accuracy", scores.mean())

0.92450066560442468

In [35]:
from sklearn.model_selection import cross_val_score
scores =cross_val_score(clf, X, y, cv=5, scoring="roc_auc") 
print ("roc auc", scores.mean())

0.68740677261372096

# Метод К соседей

In [39]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

#Проверка модели на кросс-валидации
scores = cross_validation.cross_val_score(knn, X, y, cv = 5)
print ("accuracy",scores.mean())

scores =cross_val_score(knn, X, y, cv=5, scoring="roc_auc") 
print ("roc auc", scores.mean())

('accuracy', 0.91988912240562704)
('roc auc', 0.53895909799142494)


# SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC()

#Проверка модели на кросс-валидации
scores = cross_validation.cross_val_score(svc, X, y, cv = 5)
print ("accuracy",scores.mean())

scores =cross_val_score(svc, X, y, cv=5, scoring="roc_auc") 
print ("roc auc", scores.mean())

# Random Forest

In [37]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=11)

#Проверка модели на кросс-валидации
scores = cross_validation.cross_val_score(rf, X, y, cv = 5)
print ("accuracy",scores.mean())

scores =cross_val_score(rf, X, y, cv=5, scoring="roc_auc") 
print ("roc auc", scores.mean())

('accuracy', 0.92474994503259467)
('roc auc', 0.67685586493011951)


# GBT

In [36]:
#GBT – еще один метод, строящий ансамбль деревьев решений. 
#На каждой итерации строится новый классификатор, аппроксимирующий значение градиента функции потерь.

from sklearn import ensemble
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)

#Проверка модели на кросс-валидации
scores = cross_validation.cross_val_score(gbt, X, y, cv = 5)
print ("accuracy",scores.mean())

scores =cross_val_score(gbt, X, y, cv=5, scoring="roc_auc") 
print ("roc auc", scores.mean())

('accuracy', 0.92471876325789792)
('roc auc', 0.73330629662216162)
